# Week 1: Projectile Motion using Euler's Method

In [2]:
import numpy as np
import matplotlib.pyplot as plt

# Euler Method

In [3]:
# ---------- Simple projectile motion using Euler's method ---------- #

def proj_euler(speed_m, angle, tau, dimTau=False):

    ''' This code takes dimensional tau values and
    non dimensionalises them using Ts. 
    dimTau = True means the value provided is dimensional while
    dimTau = False means the value provided is non-dimensional '''

    # Dimensionalisation parameters
    G = 9.8  # Acceleration due to gravity (m/s^2)
    Ls = 1  # Choice for scaling length (m)
    Ts = np.sqrt(Ls / G)  # Scale for time (s)

    if dimTau == True:
        tau = tau/Ts # makes tau non dimensional!

    # Convert angle to radians
    angle = np.radians(angle)

    # Non-dimensionalise initial speed
    speed = speed_m / (Ls / Ts)

    # Row vectors for non-dimensional position and velocity
    pos = np.array([0, 0])
    vel = speed * np.array([np.cos(angle), np.sin(angle)])

    # Store Initial Condition (for plotting):
    x, y = [pos[0]], [pos[1]]

    # ---- Euler's method ---- #
    
    while pos[1] >= 0:
        
        # Compute one step of Euler's method:
        # First update position using current velocity
        pos = pos + tau * vel
        # Then update velocity with gravity
        vel = vel + tau * np.array([0, -1])

        # Store position for plotting
        x.append(pos[0])
        y.append(pos[1])

    # ---- Estimate Range ---- #

    # Linear interpolation to estimate the range of the projectile
    coOrdsOver = np.array([x[-2], y[-2]])  # Last point projectile above axis
    coOrdsUnder = np.array([x[-1], y[-1]])  # Projectile under ground
    range = coOrdsUnder[0] - coOrdsUnder[1] * (coOrdsUnder[0] - coOrdsOver[0]) / (coOrdsUnder[1] - coOrdsOver[1])
    range_m = Ls * range  # Convert back to m

    # Analytic expression for range
    an_range_m = (speed_m**2 * np.sin(2 * angle)) / G

    return x, y, range_m, an_range_m


In [4]:
# First, define new method 

def proj_midpoint(speed_m, angle, tau, dimTau = False):

    '''
    dimTau = True means the value provided is dimensional while
    dimTau = False means the value provided is non-dimensional
    
    Since this code works with non-dimensional values of tau,
        if your tau is dimensional, toggle dimTau = True
        if your tau is already non-dimensional, leave the default dimTau = False '''

    # Dimensionalisation parameters
    G = 9.8  # Acceleration due to gravity (m/s^2)
    Ls = 1  # Choice for scaling length (m)
    Ts = np.sqrt(Ls / G)  # Scale for time (s)

    if dimTau == True:

        tau = tau/Ts

    # Convert angle to radians
    angle = np.radians(angle)

    # Non-dimensionalise initial speed
    speed = speed_m / (Ls / Ts)

    # Row vectors for non-dimensional position and velocity
    pos = np.array([0, 0])
    vel = speed * np.array([np.cos(angle), np.sin(angle)])

    # Store Initial Condition (for plotting):
    x, y = [pos[0]], [pos[1]]

    # ---- Midpoint method ---- #
    
    while pos[1] >= 0:
        
        # Compute one step of midpoint method:
        velp = vel  # Store value of velocity
        vel = velp + tau * np.array([0, -1])
        pos = pos + 0.5 * tau * (vel + velp)  

        # Store position for plotting
        x.append(pos[0])
        y.append(pos[1])

    # ---- Estimate Range ---- #

    # Linear interpolation to estimate the range of the projectile
    coOrdsOver = np.array([x[-2], y[-2]])  # Last point projectile above axis
    coOrdsUnder = np.array([x[-1], y[-1]])  # Projectile under ground
    range = coOrdsUnder[0] - coOrdsUnder[1] * (coOrdsUnder[0] - coOrdsOver[0]) / (coOrdsUnder[1] - coOrdsOver[1])
    range_m = Ls * range  # Convert back to m

    # Analytic expression for range
    an_range_m = (speed_m**2 * np.sin(2 * angle)) / G

    return x, y, range_m, an_range_m


In [5]:
# Dimensional time-step, tau
tau = 0.5

speed_m_new = 50
angle_new = 40

_, _, Range, Ranalytic = proj_midpoint(speed_m_new, angle_new, tau, dimTau = True)
percError = abs(Range - Ranalytic) / Ranalytic * 100

print(f'Percentage Error: {percError:.4f}%')

Percentage Error: 0.0572%


In [ ]:
time_steps = np.array([0.001, 0.01, 0.1, 1]) # non-dimensional time steps

percError_array = np.zeros(len(time_steps))

for i in range(len(time_steps)):

    tau = time_steps[i]

    _, _, Range, Ranalytic = proj_midpoint(speed_m_new, angle_new, tau)
    percError_array[i] = abs(Range - Ranalytic) / Ranalytic * 100

plt.loglog(time_steps, percError_array, 'o-k')
plt.xlabel('Time step, non dim.')
plt.ylabel('Percentage Error')
plt.show()

In [7]:
# Non dimensional time-step, tau
tau = 0.1

speed_m = 50 # Initial speed in m/s; change if needed
angle = 40 # Initial angle in degrees; change if needed

# Run the Euler simulation
x, y, range_m, an_range_m = proj_euler(speed_m, angle, tau, dimTau=False)

print(f"Range from Euler's method: {range_m:.2f} m")

Range from Euler's method: 252.45 m


In [8]:
percErrorEuler =  100 * (range_m - an_range_m) / an_range_m
print(f"Percentage error: {percErrorEuler:.4f} %")

Percentage error: 0.4865 %


In [9]:
tau = 0.1 # non dimensional tau

# Dimensionalisation parameters
G = 9.8  # Acceleration due to gravity (m/s^2)
Ls = 1  # Choice for scaling length (m)
Ts = np.sqrt(Ls / G)  # Scale for time (s)

tau_dim = tau*Ts  # Dimensional time step

print(f"Dimensional tau: {tau_dim:.4f}")

Dimensional tau: 0.0319


In [ ]:
time_steps = np.array([0.001, 0.01, 0.1, 1]) # Dimensional

numerical_range = np.zeros(len(time_steps))
percErrorEuler_array = np.zeros(len(time_steps))

for i in range(len(time_steps)):

    tau = time_steps[i]

    _, _, numerical_range[i], Ranalytic = proj_euler(speed_m, angle, tau, dimTau = True)
    percErrorEuler_array[i] = abs(numerical_range[i] - Ranalytic) / Ranalytic * 100

print("{:<10} {:<20} {:<15}".format("tau", "numerical range (m)", "percent error (%)"))
print("-" * 45)
for i in range(len(time_steps)):
    print("{:<10.4f} {:<20.6f} {:<15.6f} ".format(
        time_steps[i], numerical_range[i], percErrorEuler_array[i]))